#  Tutorial: Building a Q-Learning agent with pure Python

---

Today you'll learn the foundations of Q-Learning by creating and training your own advanced tic-tac-toe agent. You can write the code for yourself or refer to the answer functions provided here to come away with an understanding of how basic reinforcement learning algorithms adapt to different environments. For those with no coding experience, check out the version of this article with higher-level explanations and pre-written functions [here](https://github.com/js-fitz/Q-Learning/).


#### Sections:

>### **1. Setting Up the Environment**
>### **2. Making a Move**
>### **3. Simulations & Backpropagation**
>### **4. Training & Testing the Agent**

This project requires the following modules, so start here:

In [ ]:
import time
import math
import random

import ql_functions as answers # animations for tutorial
from IPython.core import display # for gameplay in Jupyter

You'll find the answers for all the prompts in this tutorial in the `Q-Learning (Answers)` notebook, which you can find [here](https://github.com/js-fitz/Q-Learning/). We use the `ql_functions` module aliased as `answers` here to display animations, to demonstrate the desired result of each function, and to provide you with an interactive gaming environment  at the end to play against your agent. Before trying each function, run the **Results** cell below it to see the desired result. Run the following block now to enable animations in the notebook:

In [ ]:
%matplotlib notebook

### Q-Learning Algorithms: An Overview

Q-Learning is a technique for teaching an agent how to respond to different states of an environment by simulating actions and evaluating their outcomes. Over the course of many games, **backpropagation** assigns penalties and rewards to particular moves based on how the game ended. Our goal is for the agent to discover the best possible action from any given state. Take a look at the following chart for an overview of the process:

In [2]:
answers.ql_flow()

Let's get started.

## 1. Setting up the environment

We will not be training our agent to hold a pencil, so first you need to generate a digital tic-tac-toe board. We may think of tic-tac-toe in two dimensions, but for your agent's purposes you can store the board in single-axis object representing board positions 1 through 9.

### 1.1 Generate a blank board

**Prompt:** Define a `new_board` function that accepts `size` as an argument and returns a list of empty tiles (`' '`) of length `size**2`.


**Answer:**

In [ ]:
def new_board(size=3):
    
    # ...
    
    return board

**Result:**

In [ ]:
answers.new_board()

### 1.2 Display the board

While the agent doesn't mind working in one dimension, a human user will tend to think of tic-tac-toe in terms of columns and rows.

**Prompt:** Create a function called `show` to print the board on a square grid. The function should accept one argument, `board`.

- Try including the `—` and `|` characters to build a grid for the board.


- **Hint:** The `sqrt` function in the `math` module will be helpful for defining the length of each row. At the end of each row, insert a line break.


- **Challenge:** Add an optional boolean variable called `helpers` to overlay the position index of empty cells (this will make it easier for a human user to make moves later on).
   - **Hint:** Use `.copy()` to avoid overwriting the board (a list reference in python is only an alias).
   

**Answer:**

In [ ]:
def show(b, helpers=False):
    
    
    # ...
    

    return

**Result:**

In [ ]:
b = answers.new_board()

answers.show(b, helpers=True)

### 1.3 Evaluating Board States

Next you need a function for evaluating the current board state, implicitly establishing some of the rules governing our environment. 

**Prompt**: Create a function called `evaluate` that accepts `b`, a board, and returns one of four possible results in the form of a string:

1. `'X Wins!'`


2. `'O Wins!'`


3. `'Continue'` (no winner, empty spaces remain)


4. `'Tie!'` (no winner, no empty spaces)


- **Hint 1:** Don't be afraid to use multiple nested `for` loops to check for a winner across distinct methods of winning
- **Hint 2:** Given the total length of the board, you may find the `sqrt` function helpful again to define the number of rows/columns.


**Answer:**

In [ ]:
def evaluate(b):
    
    
    # ...
    
    
    return

**Result:**

In [ ]:
for b in ['X OXXO OX', '  OXX  OX', 'OXOXXOXOX']:
    answers.show(b)
    print(answers.evaluate(b)) # <- put your evaluate() function here

Nice work! With the boilerplate up and running, it's time to start building your agent.


## 2. Making a move

How does an agent "decide" where to move? To contain the answer to this question we need a **Q Table**, a dictionary for the simulator to store thousands of possible states along with the Q values for different actions taken from each state. To start, the Q Table will be empty. But with a little luck, over time the action-reward pairs comprising your table will increasingly come to represent the best moves from each board state.

In the end, we want an agent who is equally adept at as playing X (going first) and O (going second), so you need to train for both cases. Think about the following question for a few minutes before reading on:

#### How can we store the Q values for both X and O in a single Q Table?


---


By flipping the player tiles for half the process, we provide the agent with a standardized language for backpropagation and for selecting moves — in other words, seeing itself as X all of the time. So whenever you reference the Q Table for a move by O, you will need a function to flip board.


### 2.1 Q Table Translator

**Prompt:** Write a function that accepts the board **as a string** (`b_key`) and returns the string with all of the Xs and Os reversed.

- **Hint:** Using `.upper()` can make the process a bit more elegant.

**Answer:**

In [ ]:
def flip_board(b_key): 
    
    # ...
    
    return b_key

**Result:**

In [ ]:
answers.flip_board('XX OOO XX')

### 2.2 The Exploration-Exploitation Trade-Off

One challenge in reinforcement learning comes in finding a balance between **exploration** and **exploitation**. Avery Parkinson illustrates with this analogy:

> Let’s say that you and your friends are trying to decide where to eat. In the past, you’ve always gone to a Mexican restaurant around the corner, and you’ve all really enjoyed it. However, this time, one of your friends mentions that a new Lebanese place has opened up down the street, and it’s supposed to be really good. None of you guys can come to a consensus — should you go to the Mexican restaurant which you know to be really good, or should you try the Lebanese place which has the potential to be better or worse? ([source](https://medium.com/analytics-vidhya/the-epsilon-greedy-algorithm-for-reinforcement-learning-5fe6f96dc870))

Similarly, our agent needs to decide throughout a simulation whether to explore new moves through random selection or to exploit prior information by choosing the move with the highest Q. We pursue this balance over many simulated decisions in Q-Learning using an **epsilon-greedy algorithm**. With epsilon (**ε**) set to 1, the agent moves completely randomly (exploring), while decreasing **ε** to zero triggers more moves based on prior knowledge (exploiting).



\begin{eqnarray*}
&&\mbox{ with probability } \epsilon: \mbox{choose an action at random}\\
&&\mbox{ with probability } 1- \epsilon: action = \max_{reward} Q(state, action\ space)
\end{eqnarray*}

If we set **ε** to shrink over time, the agent will do more exploring early on while gaining confidence as it plays more games, gradually increasing the relative number of decisions informed by the Q Table. The following simulation demonstrates the desired effect over 100 moves. Observe how different climbs down the epsilon ladder affect the total number random moves over time by changing `e_init` and `e_terminal` to any values between 0–1


In [ ]:
_= answers.simulate_e_greedy( e_init = .7,
                              e_terminal = 0 )

Your agent will use a function called `get_move` to pick either a random move or a move informed by the Q-Table in every step. Start by assigning a location for your Q Table in memory.

**Prompt:** Initialize an empty dictionary called `q_table`.

**Answer:**


**Result:**

In [ ]:
answers.q_table

Well done!


Now for a surprise: in this tutorial, you're actually going to learn *Double* Q-Learning. We'll come back to the epsilon-greedy formula shortly.


**Double Q-Learning** uses a parallel set of Q Tables to improve performance by randomly selecting one of the two tables to update at each step in backpropagation. For more on the purposes and the mathematical foundations of Double Q-Learning, see [this article](https://towardsdatascience.com/double-q-learning-the-easy-way-a924c4085ec3). For now, it's important to know that each value in your Q Table will actually hold **two nested dictionaries** of `action:reward` pairs called `Q1` and `Q2`. After training, your `q_table` should look something like this:

```
q_table = {
         ...
    'OXOXO    ': { Q1: {5: 0.32, 6: 0.23, 7: 0.35, 8: 0.65}
                   Q2: {5: 0.13, 6: 0.27, 7: 0.29, 8: 0.65} }
    'XOXOXO   ': { Q1: {6: 0.27, 7: 0.49, 8: 0.16}
                   Q2: {6: 0.74, 7: 0.29, 8: 0.63} }
         ...
          }
```


Each key in the Q Table represents a board state containing two nested dictionaries `Q1` and `Q2`, representing both versions of the Q Table. For every state, both versions of Q values have available moves as keys, and associated rewards for each move as values.  Keep this format for a Double Q Table in mind for following sections. Consider this updated diagram to recap:

![Double Q Learning](dql.png)

This means in your e-greedy move function, to make a "smart" move get the **average** Q values from both tables. Remember we set the Q Table as a global variable, so `get_move` will interact with our global name space to determine these values. Now finally, we're ready to make a move.

### 2.3 ε-Greedy Move Function

**Prompt:** Create a function called `get_move` to accept three default arguments: `b` (the current board), `epsilon`, and `player` (`X` or `O`). Using the epsilon-greedy algorithm below, the function should select and return the next move in the form of an index of an open position on the board. We outline every step of the function for you here:

1. Start by identifying unnocuppied positions on the board, listing all possible moves 


2. If the player is 'O', reverse the tiles on the board using the `flip_board` function you wrote above


3. If the state does not already exist in the `q_table`, initialize it with a nested dictionary containing two sets of `action:reward` pairs named `Q1` and `Q2`. The keys in both versions of the  table should be a list of all possible moves from the given state. Initialize their values at **0.6 or higher** in new states (not already in the Q Table) for **optimistic Q-learning** — confidence in new moves promotes exploration. Include `init_q` as a new keyword argument in your `get_move` function.


4. Use the following ε-greedy formula in conjunction with the `random` module to pick a move strategy:

\begin{eqnarray*}
&&\mbox{ with probability } \epsilon: \mbox{choose an action at random}\\
&&\mbox{ with probability } 1- \epsilon: action = \max_{reward} Q(state, action\ space)
\end{eqnarray*}

5. For an "intelligent" move, pick the position with the highest average reward from **both** versions of the Q Table. Early on in training, this value will often be equal or close to your initial Q value.
    - **Challenge:** Get the average Q values from both nested dictionaries in one line of code.


6. To test your function, make sure it initializes new board states in the `q_table` in the format of the example above.  


**Answer:**

In [ ]:
def get_move(b, epsilon, player, init_q=.6):
    
    
    # ...
    
    
    return move

**Result:**

Run `get_move` on a blank board, then check out your `q_table` to make sure the state got initialized.

In [ ]:
answers.get_move(answers.new_board())

If you decrease the epsilon value in `get_move` and run it a few times, position `0` should become significantly overfavored. That's because our Q values are all equal right now at the initial Q value, so the position with the highest Q value is simply the first in the list.

## 3. Simulations and Backpropagation

### 3.1 Match simulator

To populate your Q Table with better values, you'll need a function to simulate an entire game. This function should start by creating a fresh board, then iterate back and forth between players using `get_move` to update the board at each step until reaching a win or a tie.

**Prompt:** Define a function called `simulate_game` that accepts two keyword arguments, `epsilon_x` and `epsilon_o`. With X always going first, plug the respective players' **ε** values into your move function for each player and keep updating the board until reaching a terminal state.

1. Use your `evaluator` function to check the board at the beginning of each iteration
    - As long as the `result` of `evaluator` contains `C`, players should keep making moves using their respective values for epsilon.


2. Backpropagation requires that we **remember the sequence of moves in a game**, so at the end we can go back and adjust the Q value for each action with either a reward or a penalty depending on the match result.
   - Before starting the game loop, create an empty list called `steps`.
   - For every move, append a two-item dictionary to `steps` containing the given board `state` (as a 9-character string) and the next `move`.
   
   
3. Include an additional boolean keyword argument called `verb` to print the result and `show` the final board state. 
   
The function should return `steps`—a list of dictionaries with the current `state` and the next `move` at each step—and `winner`, a singe character representing the final result: `X` or `O` for a win and `T` for a tie.

---

**Answer:**

In [ ]:
def simulate_game(epsilon_x=1, epsilon_o=1, verb=False):
    
    
    # ...  
    
    
    return steps, winner

**Result:**

In [ ]:
steps, winner = answers.simulate_game(slow_down=4)
steps

Excellent. Our agent can now play a full (albeit entirely random) match.

Here is where backpropagation comes in: using the `steps` object, you will go back and update the Q table by assigning rewards and penalties to particular moves. This is the final and perhaps most crucial step to any reinforcement learning algorithm: through backpropapagation, over time the agent will "learn" various paths to success as well as those leading to failure.

### 3.2 Updating Q-Values

When you send the algorithm back into the sequence of steps in a match, for each action you will need a function to estimate a new Q value. The formula for updating Q is as follows:

\begin{eqnarray*}
Q_\mbox{action} = (1-\mbox{learning rate})* Q_\mbox{action} + \mbox{learning rate}*(reward_\mbox{action} + discount * \mbox{max_future_q})\\
\end{eqnarray*}

In essence, we adjust the reward a one move from a given state by accounting both for the outcome of the game (`reward`) **and** for the estimated best **next move** possible from future states after taking the current action (`max_future_q`). The second factor asks: did this move set up a good future move? Two global variables set the bounds for how these factors affect the Q Table:


    
- Learning Rate — `lrate` — determines how aggressively to adjust Q values   
   
- Discount Factor — `discount` — determines the importance of future rewards

**Prompt:** Set these two **global** variables equal to `.15` and `.9` respectively.

In [ ]:
lrate = 
discount = 

You'll also feed the Q value updater function three **local** (step-specific) variables:
    
- `current_q` (float) — Initialized at .6, over time these values should start to reflect the quality of the moves
   
- `reward` (float) — Depends on how many moves remain until the end of the game (will be determined in the next function)
    
- `max_future_q` (float) — The Q value of the best possible next move according to the current Q Table (not necessarily a move made in this game).  
   

**Prompt:** Using the Q value update formula above, create a function called `get_new_q` that accepts these three variables and returns a new Q value.


---


**Answer:**

In [ ]:
def get_new_q(current_q, reward, max_future_q):
    
    # ...
    
    return new_q

**Result:**

In [ ]:
answers.get_new_q(current_q=.6, reward=.43, max_future_q=.9)

Great work! You're almost done.

Before moving on to the final step, review the `steps` output from the `simulate_game` function.

In [ ]:
answers.simulate_game()[0]

For backpropagation, your agent needs to go back into the moves made by each player after the game is over. How can you index the states in `steps` where only one player made their moves? Consider how the outcome of a game affects the player who moved last.

### 3.3. Backpropagation

**Prompt (Challenge):** Define a function called `backpropagate` to accept three arguments: `steps` and `winner` (the outputs of `simulate_game`) and `alpha` (a constant). The function should iterate backwards through `steps`, calling `get_new_q` for the move made from each `state`. `backpropagate` will draw from and make changes to your global `q_table` dictionary.

The only variable we have yet to define is `reward`. Using the following formula, we assign greater rewards to moves made later in the game. Set the default `alpha` value to `.9` in your `backpropagate` function.

\begin{eqnarray*}
&&\mbox{reward} =  alpha ^\mbox{(n steps remaining)}\\
\end{eqnarray*}


If the player ultimately lost, make the reward negative to reflect a penalty. In the case of a tied game, consider setting the reward to a negative value but also shrinking its magnitude to make hard losses worse in comparison. All the necessary steps for `backpropagate` are outlined below:


1. After isolating the steps made by a given player, iterate backwards through their moves. For each `state` where `O` makes a move, remember to reverse the board tiles before referencing the `q_table`.


2. Use the `random` module to pick either `Q1` or `Q2` for **each move**. This table should become the target of your update on this `state`.


3. Recall that `max_future_q` is the highest possible Q value from the **next state**. Technically this means looking forward two steps, to when the same player had another chance to move.
    - For a player's final move, set the `max_future_q` value manually. For example, 1 for a win, .5 for a tie and 0 for a loss.
    - The `max_future_q` and `current_q` should be determined based on the randomly selected target version of your `q_table` in each step.


4. Per the reward formula above, you may find it helpful to `enumerate` the list of states in which a given player made their moves in order to count how many steps remain from a given state. Work with a copy of the original `steps` object when doing so to to avoid losing data.
   - **Hint:** Indexing for a player's moves takes more than just a `[::2]` reversal — consider how different game results alter the `steps` object. 
   - Remember to make the reward negative if the player lost
   - If the game ended in tie, consider shrinking the reward or penalty. For example, you can multiply it by 0.1.
   

5. After running `get_new_q`, make sure to write the `new_q` value directly into the target version (`Q1` or `Q2`) of your global `q_table`.

In [ ]:
def backprop(steps, winner, alpha=.9, x_verb=False, o_verb=False ):  
        
    
    # ...
    
    
    return

---

**Result:**

To better illustrate the process, we included extra print statements in this function to show what happens in each step.

In [ ]:
print('Final result:')
steps, winner = answers.simulate_game(verb=True)
answers.backpropagate(steps, winner, wait_seconds=2.5)

## 4. Training & Testing the Agent

Now comes the most rewarding part of reinforcement learning: training your agent on a series of simulations. Get creative with your training technique, remembering that we have explicit control over the epsilon values for each player thanks to the keyword arguments you built into `simulate_game`.


### 4.1 Training
    
    
**Prompt:** Write a function called `train_agent` that accepts an argument `iters`, defining the number of games per training session. At the end of each game, the function should `backprop` over the match `steps` and `winner` to update the global `q_table` dictionary.

1. To encourage more exploration in the early parts of your simulation, set a high initial epsilon value and decrease it over time.
    - **Hint:** Given that you can control the epsilon value for each player, consider training in batches where one or both players move completely randomly. Training in this fashion can help your agent become more robust to strange moves.


2. **Challenge:** Calculate the players' winning rates at the end of every game in order to visualize the learning process. During or after the simulation, `train_agent` should return these stats or plot them.


3. Don't forget to call `backprop` at the end of every game.


---

**Answer:**

In [ ]:
def train_agent(iters=1000):
    
    
    # ...
    
    

**Results:**

There are many ways to approach the problem of training, so we provide functions below for you to visualize two possible strategies. Both of our `train_agent` functions are wrapped in plotting scripts, to depict each players' winning rates over time as well as the shrinking epsilon values.

The first function, `visualize_learning`, animates the players' winning rates over time. To the right of the winning rate plots, we also include heatmaps for the Q values in two select boards. Here you can see how the agent's knowledge about a particular state develops over time.

In [ ]:
_=answers.visualize_learning()

Observe the evolving Q values for actions from `'         '` — a blank board. An empty board is the only state guaranteed to appear in every game, so it recieves an update every time we `backpropagate`. A heatmap for `'    X    '` is also included to visualize O's response if X starts on position 5. Run the simulation a few times to see both players improve with more practice.

Math tells us there are [4,520](https://math.stackexchange.com/questions/3276315/number-of-uncompleted-tic-tac-toe-games) possible incomplete states on a 3x3 tic-tac-toe boarrd, i.e. states we our agent to learn about. After playing a few hundred games, how many of these states has our agent seen? Replace the `q_table` from the `answers` module with your global `q_table` in this notebook to see how your training stacks up.  

In [ ]:
print(f'The agent has encountered {round(100*len(answers.q_table)/4520, 2)}% of all possible board states on a 3x3 board')

To bring this number closer to 100, try to clean up your code to make it work faster, then run many more simulations. Around 50,000 games seems to work.

In [ ]:
_=answers.full_training()

How many board states has our agent seen now?

In [ ]:
print(f'The agent has encountered {round(100*len(answers.q_table)/4520, 2)}% of all possible board states on a 3x3 board')

### 4.2 Testing

The following code is set up for you to play with the agent in an interactive game environment. With your agent sufficiently trained, give it your best shot.

Playing as `X` or as `O`, can you outsmart the Q Table?

In [ ]:
answers.versus_agent() # agent from answers module

# Use this to play against the agent you built:
# answers.versus_agent(custom_q_table = q_table) 

---

**Bonus:** Increase the size of your environment by playing a bigger board. Use our functions below to experiment, and prepare to sit back for a minute while the agent adapts to the more complex environment with a longer training session. Run the following few cells, then play as O against the agent: it should be able to win every game.

In [ ]:
answers.game_settings(size=4,
                     min_contig=3)

In [ ]:
_=answers.full_training(batches=25)

In [ ]:
answers.versus_agent()